In [2]:
import pandas as pd
import random
import os
import requests
import time
from datetime import datetime

# --- Cấu hình Moodle API ---
MOODLE_URL = 'http://localhost:8100/webservice/rest/server.php'
TOKEN = '84cffdbb9ead18d97ccc45f9889bc926'
FORMAT = 'json'

# --- Hằng số và siêu tham số cho Q-learning ---
LEARNING_RATE = 0.1
DISCOUNT_FACTOR = 0.9
EXPLORATION_RATE = 0.2

ACTIONS = [
    'read_new_resource',
    'review_old_resource',
    'attempt_new_quiz',
    'redo_failed_quiz',
    'skip_to_next_module'
]

READ_RATE_BINS = [0.37, 0.69]

q_table = {}

# --- Hàm gọi API Moodle ---
def call_api(function_name, params):
    params.update({
        'wstoken': TOKEN,
        'wsfunction': function_name,
        'moodlewsrestformat': FORMAT
    })
    try:
        response = requests.get(MOODLE_URL, params=params)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"❌ API {function_name} lỗi: {e}")
        return {}

# --- Các hàm cho Q-learning và xử lý dữ liệu ---
def load_q_table_from_csv(filename='q_table_results.csv'):
    q_table_loaded = {}
    if not os.path.exists(filename):
        return {}
    
    df = pd.read_csv(filename)
    for _, row in df.iterrows():
        state = (row['sectionid'], row['read_bin'], row['level'])
        action = row['action']
        q_value = row['q_value']
        
        if state not in q_table_loaded:
            q_table_loaded[state] = {a: 0.0 for a in ACTIONS}
        
        q_table_loaded[state][action] = q_value
    return q_table_loaded

def save_q_table_to_csv(q_table_to_save, filename='q_table_results.csv'):
    q_table_data = []
    for state, actions_dict in q_table_to_save.items():
        for action, q_value in actions_dict.items():
            row = {
                'sectionid': state[0],
                'read_bin': state[1],
                'level': state[2],
                'action': action,
                'q_value': q_value
            }
            q_table_data.append(row)
    
    df = pd.DataFrame(q_table_data)
    df.to_csv(filename, index=False)

def initialize_q_table_with_clusters(filename='q_table_results.csv', cluster_file='./synthetic_user_features_clustered.csv'):
    initial_q_table = {}
    try:
        df_clusters = pd.read_csv(cluster_file)
        unique_clusters = df_clusters['cluster'].unique()
    except FileNotFoundError:
        unique_clusters = [0]

    read_bins_range = range(len(READ_RATE_BINS) + 1)
    quiz_passed_states = [0, 1]

    if os.path.exists('./user_insight.csv'):
        df_insight = pd.read_csv('./user_insight.csv')
        unique_sectionids = df_insight['sectionid'].unique()
    else:
        unique_sectionids = [1, 2, 3, 4, 5]

    for r_bin in read_bins_range:
        for q_passed in quiz_passed_states:
            for cluster in unique_clusters:
                for section_id in unique_sectionids:
                    state = (int(section_id), r_bin, q_passed)
                    initial_q_table[state] = {action: 0.0 for action in ACTIONS}
    
    save_q_table_to_csv(initial_q_table, filename)
    return initial_q_table

def get_user_state_from_insight(user_id, course_id, section_id):
    try:
        df = pd.read_csv('./user_insight.csv')
        user_insight = df[(df['userid'] == user_id) & (df['courseid'] == course_id) & (df['sectionid'] == section_id)]
        
        if user_insight.empty:
            return None
        
        user_insight = user_insight.sort_values(by='time', ascending=False)
        latest_record = user_insight.iloc[0]
        
        completion_rate = latest_record['completion_rate']
        bin_resource = 0
        if completion_rate > READ_RATE_BINS[0] and completion_rate <= READ_RATE_BINS[1]:
            bin_resource = 1
        elif completion_rate > READ_RATE_BINS[1]:
            bin_resource = 2

        state = (
            int(latest_record['sectionid']),
            bin_resource,
            int(latest_record['quiz_passed'])
        )
        return state
        
    except FileNotFoundError:
        return None
    except Exception as e:
        print(f"❌ Lỗi khi đọc file user_insight.csv: {e}")
        return None

def get_latest_moodle_event():
    file_path = './user_insight.csv'
    if not os.path.exists(file_path):
        return None
    
    try:
        df = pd.read_csv(file_path)
        if df.empty:
            return None
        
        df = df.sort_values(by='time', ascending=False)
        latest_record = df.iloc[0]
        
        return {
            'user_id': int(latest_record['userid']),
            'course_id': int(latest_record['courseid']),
            'section_id': int(latest_record['sectionid'])
        }
    except Exception as e:
        print(f"❌ Lỗi khi đọc file user_insight.csv: {e}")
        return None

def get_q_value(state, action):
    global q_table 
    if state not in q_table:
        q_table[state] = {a: 0.0 for a in ACTIONS}
    return q_table[state][action]

def update_q_table(state, action, reward, next_state):
    global q_table
    current_q = get_q_value(state, action)
    
    if next_state not in q_table:
        max_future_q = 0.0
    else:
        max_future_q = max(q_table[next_state].values())

    new_q = current_q + LEARNING_RATE * (reward + DISCOUNT_FACTOR * max_future_q - current_q)
    q_table[state][action] = new_q

def get_reward(current_state, next_state):
    reward = 0
    if current_state[2] == 0 and next_state[2] == 1:
        reward += 10
    if next_state[1] > current_state[1]:
        reward += 5
    if next_state[0] > current_state[0]:
        reward += 7
    if next_state[0] < current_state[0]:
        reward -= 5
    return reward

def suggest_next_action(current_state, q_table):
    if current_state not in q_table or random.uniform(0, 1) < EXPLORATION_RATE:
        action = random.choice(ACTIONS)
        q_value = get_q_value(current_state, action)
        return action, q_value
    else:
        q_values = q_table[current_state]
        best_action = max(q_values, key=q_values.get)
        best_q_value = q_values[best_action]
        return best_action, best_q_value

def handle_moodle_event():
    global q_table
    
    event_data = get_latest_moodle_event()
    if not event_data:
        print("Không có sự kiện Moodle mới để xử lý.")
        return

    user_id = event_data['user_id']
    course_id = event_data['course_id']
    section_id = event_data['section_id']

    current_state = get_user_state_from_insight(user_id, course_id, section_id)
    if current_state is None:
        return

    suggested_action, q_value = suggest_next_action(current_state, q_table)
    print(f"Đề xuất hành động tiếp theo cho người dùng {user_id} là: '{suggested_action}' với Q-value là {q_value:.2f}")

    # Chờ để dữ liệu mới được ghi vào user_insight.csv sau khi hành động đã thực hiện
    time.sleep(5)
    
    next_state = get_user_state_from_insight(user_id, course_id, section_id)
    if next_state is None:
        return

    reward = get_reward(current_state, next_state)
    update_q_table(current_state, suggested_action, reward, next_state)
    save_q_table_to_csv(q_table)

# --- Điểm khởi chạy của chương trình ---
if __name__ == '__main__':
    q_table = load_q_table_from_csv()
    if not q_table:
        q_table = initialize_q_table_with_clusters()

    handle_moodle_event()

Đề xuất hành động tiếp theo cho người dùng 4 là: 'read_new_resource' với Q-value là 0.00
